In [ ]:
#Packages
import os
import pandas as pd
import numpy as np
import xgboost
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score, roc_curve
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
import matplotlib.pyplot as plt
import seaborn as sns
import shap

In [ ]:
#Get Data
csv_dir = '../../SEC Trackman Data/'
all_files = [os.path.join(csv_dir, f) for f in os.listdir(csv_dir) if f.endswith('.csv')]

df_list = []

for file in all_files:
    df = pd.read_csv(file)
    df_list.append(df)

all_pitches = pd.concat(df_list, ignore_index=True)

print(all_pitches.shape)
print(all_pitches.columns)

In [ ]:
#Get all fastballs
fb = all_pitches[all_pitches['TaggedPitchType'].isin(['Fastball','Cutter','Sinker','FourSeamFastBall','TwoSeamFastBall','OneSeamFastBall'])]
print(fb.shape)

In [ ]:
#Subset fastballs into platoon splits
fb_rr = fb[(fb['PitcherThrows'] == 'Right') & (fb['BatterSide'] == 'Right')]
fb_rl = fb[(fb['PitcherThrows'] == 'Right') & (fb['BatterSide'] == 'Left')]
fb_lr = fb[(fb['PitcherThrows'] == 'Left') & (fb['BatterSide'] == 'Right')]
fb_ll = fb[(fb['PitcherThrows'] == 'Left') & (fb['BatterSide'] == 'Left')]

print(fb_rr.shape)
print(fb_rl.shape)
print(fb_lr.shape)
print(fb_ll.shape)

In [ ]:
#Whiffs
fb_rr['whiff'] = np.where(fb_rr['PitchCall']=='StrikeSwinging',1,0)
fb_rl['whiff'] = np.where(fb_rl['PitchCall']=='StrikeSwinging',1,0)
fb_lr['whiff'] = np.where(fb_lr['PitchCall']=='StrikeSwinging',1,0)
fb_ll['whiff'] = np.where(fb_ll['PitchCall']=='StrikeSwinging',1,0)

In [ ]:
preds = ['RelSpeed','VertRelAngle','HorzRelAngle','SpinRate','SpinAxis','RelHeight','RelSide','Extension','VertBreak','InducedVertBreak','HorzBreak','ZoneSpeed','VertApprAngle','HorzApprAngle','ZoneTime','pfxx','pfxz','x0','y0','z0','vx0','vy0','vz0','ax0','ay0','az0','SpeedDrop','PitchTrajectoryXc1','PitchTrajectoryXc2','PitchTrajectoryYc0','PitchTrajectoryYc1','PitchTrajectoryYc2','PitchTrajectoryZc0','PitchTrajectoryZc1','PitchTrajectoryZc2']

Righty vs. Righty Split

In [ ]:
fb_rr_X = fb_rr[preds]
fb_rr_y = fb_rr['whiff']

In [ ]:
#Train Model: RHP vs RHB
X_train, X_test, y_train, y_test = train_test_split(fb_rr_X, fb_rr_y, test_size=.33, random_state=25)

model = XGBClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
y_pred_prob = model.predict_proba(X_test)[:,1]
roc_auc = roc_auc_score(y_test, predictions, multi_class='ovr')

print(f'ROC AUC: {roc_auc:.4f}')

# Plotting ROC Curve for one class (e.g., class 0)
fpr, tpr, _ = roc_curve(y_test == 1, y_pred_prob)
plt.plot(fpr, tpr, label='Class 0 ROC curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc='best')
plt.show()

In [ ]:
aub_fb_rr = fb_rr[fb_rr['PitcherTeam']=='AUB_TIG']
print(aub_fb_rr.shape)

In [ ]:
# Get unique values
unique_values = fb_rr['Pitcher'].unique()
aub_unique_values = aub_fb_rr['Pitcher'].unique()

print(aub_unique_values)

# Create a dictionary to store subsets
all_subsets = {value: fb_rr[fb_rr['Pitcher'] == value] for value in unique_values}

aub_subsets = {value: fb_rr[fb_rr['Pitcher'] == value] for value in aub_unique_values}

In [ ]:
def aub_rr_fb_stuff(name):
    
    preds = ['RelSpeed','VertRelAngle','HorzRelAngle','SpinRate','SpinAxis','RelHeight','RelSide','Extension','VertBreak','InducedVertBreak','HorzBreak','ZoneSpeed','VertApprAngle','HorzApprAngle','ZoneTime','pfxx','pfxz','x0','y0','z0','vx0','vy0','vz0','ax0','ay0','az0','SpeedDrop','PitchTrajectoryXc1','PitchTrajectoryXc2','PitchTrajectoryYc0','PitchTrajectoryYc1','PitchTrajectoryYc2','PitchTrajectoryZc0','PitchTrajectoryZc1','PitchTrajectoryZc2']
    
    all_preds = model.predict_proba(fb_rr_X)[:,1]
    mean_pred = np.mean(all_preds)

    pitcher = aub_subsets[name]
    pitcher_x = pitcher[preds]

    whiff_preds = model.predict_proba(pitcher_x)[:,1]

    stuff = whiff_preds/mean_pred
    
    print(f"{name} RR Cutter stuff+: {np.mean(stuff * 100)}")

In [ ]:
for pitcher in aub_unique_values:
    aub_rr_fb_stuff(pitcher)

Righty vs. Lefty Splits

In [ ]:
fb_rl_X = fb_rl[preds]
fb_rl_y = fb_rl['whiff']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(fb_rl_X, fb_rl_y, test_size=.33, random_state=25)

model = XGBClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
aub_fb_rl = fb_rl[fb_rl['PitcherTeam']=='AUB_TIG']
print(aub_fb_rl.shape)

In [ ]:
# Get unique values
unique_values = fb_rl['Pitcher'].unique()
aub_unique_values = aub_fb_rl['Pitcher'].unique()

print(aub_unique_values)

# Create a dictionary to store subsets
all_subsets = {value: fb_rl[fb_rl['Pitcher'] == value] for value in unique_values}

aub_subsets = {value: fb_rl[fb_rl['Pitcher'] == value] for value in aub_unique_values}

In [ ]:
def aub_rl_fb_stuff(name):
    
    preds = ['RelSpeed','VertRelAngle','HorzRelAngle','SpinRate','SpinAxis','RelHeight','RelSide','Extension','VertBreak','InducedVertBreak','HorzBreak','ZoneSpeed','VertApprAngle','HorzApprAngle','ZoneTime','pfxx','pfxz','x0','y0','z0','vx0','vy0','vz0','ax0','ay0','az0','SpeedDrop','PitchTrajectoryXc1','PitchTrajectoryXc2','PitchTrajectoryYc0','PitchTrajectoryYc1','PitchTrajectoryYc2','PitchTrajectoryZc0','PitchTrajectoryZc1','PitchTrajectoryZc2']
    
    all_preds = model.predict_proba(fb_rl_X)[:,1]
    mean_pred = np.mean(all_preds)

    pitcher = aub_subsets[name]
    pitcher_x = pitcher[preds]

    whiff_preds = model.predict_proba(pitcher_x)[:,1]

    stuff = whiff_preds/mean_pred
    
    print(f"{name} RL Cutter stuff+: {np.mean(stuff * 100)}")

In [ ]:
for pitcher in aub_unique_values:
    aub_rl_fb_stuff(pitcher)

Lefty vs. Righty Splits

In [ ]:
fb_lr_X = fb_lr[preds]
fb_lr_y = fb_lr['whiff']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(fb_lr_X, fb_lr_y, test_size=.33, random_state=25)

model = XGBClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
aub_fb_lr = fb_lr[fb_lr['PitcherTeam']=='AUB_TIG']
print(aub_fb_lr.shape)

In [ ]:
# Get unique values
unique_values = fb_lr['Pitcher'].unique()
aub_unique_values = aub_fb_lr['Pitcher'].unique()

print(aub_unique_values)

# Create a dictionary to store subsets
all_subsets = {value: fb_lr[fb_lr['Pitcher'] == value] for value in unique_values}

aub_subsets = {value: fb_lr[fb_lr['Pitcher'] == value] for value in aub_unique_values}

In [ ]:
def aub_lr_fb_stuff(name):
    
    preds = ['RelSpeed','VertRelAngle','HorzRelAngle','SpinRate','SpinAxis','RelHeight','RelSide','Extension','VertBreak','InducedVertBreak','HorzBreak','ZoneSpeed','VertApprAngle','HorzApprAngle','ZoneTime','pfxx','pfxz','x0','y0','z0','vx0','vy0','vz0','ax0','ay0','az0','SpeedDrop','PitchTrajectoryXc1','PitchTrajectoryXc2','PitchTrajectoryYc0','PitchTrajectoryYc1','PitchTrajectoryYc2','PitchTrajectoryZc0','PitchTrajectoryZc1','PitchTrajectoryZc2']
    
    all_preds = model.predict_proba(fb_lr_X)[:,1]
    mean_pred = np.mean(all_preds)

    pitcher = aub_subsets[name]
    pitcher_x = pitcher[preds]

    whiff_preds = model.predict_proba(pitcher_x)[:,1]

    stuff = whiff_preds/mean_pred
    
    print(f"{name} LR Cutter stuff+: {np.mean(stuff * 100)}")

In [ ]:
for pitcher in aub_unique_values:
    aub_lr_fb_stuff(pitcher)

Lefty vs. Lefty Split

In [ ]:
fb_ll_X = fb_ll[preds]
fb_ll_y = fb_ll['whiff']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(fb_ll_X, fb_ll_y, test_size=.33, random_state=25)

model = XGBClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
aub_fb_ll = fb_ll[fb_ll['PitcherTeam']=='AUB_TIG']
print(aub_fb_ll.shape)

In [ ]:
# Get unique values
unique_values = fb_ll['Pitcher'].unique()
aub_unique_values = aub_fb_ll['Pitcher'].unique()

print(aub_unique_values)

# Create a dictionary to store subsets
all_subsets = {value: fb_ll[fb_ll['Pitcher'] == value] for value in unique_values}

aub_subsets = {value: fb_ll[fb_ll['Pitcher'] == value] for value in aub_unique_values}

In [ ]:
def aub_ll_fb_stuff(name):
    
    preds = ['RelSpeed','VertRelAngle','HorzRelAngle','SpinRate','SpinAxis','RelHeight','RelSide','Extension','VertBreak','InducedVertBreak','HorzBreak','ZoneSpeed','VertApprAngle','HorzApprAngle','ZoneTime','pfxx','pfxz','x0','y0','z0','vx0','vy0','vz0','ax0','ay0','az0','SpeedDrop','PitchTrajectoryXc1','PitchTrajectoryXc2','PitchTrajectoryYc0','PitchTrajectoryYc1','PitchTrajectoryYc2','PitchTrajectoryZc0','PitchTrajectoryZc1','PitchTrajectoryZc2']
    
    all_preds = model.predict_proba(fb_ll_X)[:,1]
    mean_pred = np.mean(all_preds)

    pitcher = aub_subsets[name]
    pitcher_x = pitcher[preds]

    whiff_preds = model.predict_proba(pitcher_x)[:,1]

    stuff = whiff_preds/mean_pred
    
    print(f"{name} LL Cutter stuff+: {np.mean(stuff * 100)}")

In [ ]:
for pitcher in aub_unique_values:
    aub_ll_fb_stuff(pitcher)

In [ ]:
aub_subsets['Keplinger, Konner']